In [5]:
import numpy as np
import os
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout

In [43]:
model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape=(50, 50, 1), padding="same", kernel_initializer="orthogonal"))
model.add(Activation("relu"))
model.add(Dropout(0.1))

model.add(Conv2D(16, (3, 3), kernel_initializer="orthogonal", padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.1))

model.add(Conv2D(32, (3, 3), kernel_initializer="orthogonal", padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(7, 7)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(2, activation="softmax"))
          
model.compile(
    loss="categorical_crossentropy", 
    optimizer="adam",
    metrics=["accuracy"]
)
model.load_weights("best models/val_loss_model.048-0.3876-0.3249.h5")

In [214]:
test_img_root = "test"
img_files = os.listdir(test_img_root)
labels = {0: "Cat", 1: "Dog"}

images = []
orig_images = []

for img_file in img_files:
    img_path = os.path.join(test_img_root, img_file)
    orig_image = cv2.imread(img_path)
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (50, 50))
    
    orig_images.append(cv2.resize(orig_image, (600, 600)))
    images.append(image)
    
images = np.asarray(images)
orig_images = np.asarray(orig_images)

predictions = []

for image in images:
    image = image / 255
    image = np.reshape(image, (1, 50, 50, 1))
    prediction = model.predict(image)
    arg_max = np.argmax(prediction, axis=1)[0]
    label = labels[arg_max]
    predictions.append((label, prediction[0][arg_max]))

i = 0
cat_icon = cv2.imread("icons/cat-icon.png")
dog_icon = cv2.imread("icons/dog-icon.png")
color_map = {"Cat": (0, 255, 0), "Dog": (0, 0, 255)}
icon_map = {"Cat": cat_icon, "Dog" : dog_icon}
rect_start_x_y = (17, 15)
rect_end_x_y = (573, 571)
icon_x = 25
icon_y = 73
text_x = 30
text_y = 55 

while i < len(orig_images) and i >= 0:
    image = orig_images[i]
    image = cv2.rectangle(image, rect_start_x_y, rect_end_x_y, color_map[predictions[i][0]], 2)
    image = cv2.putText(image, predictions[i][0] + ":" + str(int(predictions[i][1] * 100)) + "%", (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color_map[predictions[i][0]], 2)
    cropped = image[icon_x:icon_y, icon_x + 480:icon_y + 480]
    image[icon_x:icon_y, icon_x + 480:icon_y + 480] = cv2.addWeighted(cropped, 0.5, icon_map[predictions[i][0]], 0.6, 0)
    
    cv2.imshow("Live Cat & Dog Prediction", image)
    
    k = cv2.waitKey()
    
    if k == ord('d'):
        i += 1
    elif k == ord('a'):
        i -= 1
    elif k == 27:
        break
        
cv2.destroyAllWindows()